<a href="https://colab.research.google.com/github/tpgus2603/DataMining/blob/main/contenbase_forintergeated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd ./drive/MyDrive/json_csv_files/amazon_review2

[Errno 2] No such file or directory: './drive/MyDrive/json_csv_files/amazone_review2'
/content/drive/MyDrive/json_csv_files


In [ ]:
cd amazon_review2/

/content/drive/MyDrive/json_csv_files/amazon_review2


'/content/drive/MyDrive/json_csv_files/amazon_review2'

In [ ]:
# 라이브러리 설치 (필요시)
!pip install pandas scikit-learn

# 라이브러리 임포트
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

여기서부터 제대로 시작!



In [ ]:
pwd

'/content/drive/MyDrive/json_csv_files'

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

# 상품 데이터 파일 경로
products_file_path = './combined_data2.json'  # 실제 경로로 변경
products_df = pd.read_json(products_file_path, lines=True)

# 원하는 카테고리 목록
desired_categories = [ 'Movies_TV', 'Grocery_Gourmet_Food', 'Electronics']

# 카테고리 필터링
filtered_products_df = products_df[products_df['category'].isin(desired_categories)].copy()
filtered_products_df = filtered_products_df[['rating', 'title', 'text', 'helpful_vote',
                                             'verified_purchase', 'product_title', 'category', 'parent_asin']]

print("Filtered Products DataFrame:")
print(filtered_products_df.head())

# user_reviews 데이터 로드
user_reviews_file_path = './user_reviews.json'  # 실제 경로로 변경
user_reviews_df = pd.read_json(user_reviews_file_path, lines=True)
user_reviews_df = user_reviews_df[['title', 'text', 'helpful_vote', 'verified_purchase', 'product_title', 'category', 'parent_asin']]
user_reviews_df = user_reviews_df[user_reviews_df['category'].isin(desired_categories)].copy()

print("User Reviews DataFrame:")
print(user_reviews_df.head())

Filtered Products DataFrame:
       rating                                              title  \
29669       5                                         Five Stars   
29670       5                                         Five Stars   
29671       3                       Some decent moments...but...   
29672       4  Decent Depiction of Lower-Functioning Autism, ...   
29673       5                                    What Love Is...   

                                                    text  helpful_vote  \
29669           Amazon, please buy the show! I'm hooked!             0   
29670                         My Kiddos LOVE this show!!             0   
29671  Annabella Sciorra did her character justice wi...             0   
29672  ...there should be more of a range of characte...             1   
29673  ...isn't always how you expect it to be, but w...             0   

       verified_purchase    product_title   category parent_asin  
29669                  1      Sneaky Pete  Movies_

In [ ]:
# test_reviews 데이터 로드
test_reviews_file_path = './test_reviews.json'  # 실제 경로로 변경
test_reviews_df = pd.read_json(test_reviews_file_path, lines=True)
test_reviews_df = test_reviews_df[['title', 'text', 'helpful_vote', 'verified_purchase', 'product_title', 'category', 'parent_asin']]
test_reviews_df = test_reviews_df[test_reviews_df['category'].isin(desired_categories)].copy()

print("Test Reviews DataFrame:")
print(test_reviews_df.head())

def combine_text(df):
    return df['title'].fillna('') + ' ' + df['text'].fillna('') + ' ' + df['product_title'].fillna('')

# test_reviews를 Train/Val로 분할 (7:3)
train_reviews, val_reviews = train_test_split(test_reviews_df, test_size=0.3, random_state=42)

# 카테고리별로 TF-IDF를 계산하기 위해 각 카테고리별로 반복 수행
# 사용자 요구사항 상, 각 카테고리별로 같은 로직 수행
rating_threshold = 5.0

Test Reviews DataFrame:
                                               title  \
0  The chronicles of narnia the lion, witch and w...   
1                                     trading places   
2                                          Bad Santa   
3  The chronicles of narnia: the voyage of the da...   
4                                        A bugs life   

                                                text  helpful_vote  \
0  7 year old grandson is very into the old serie...             2   
1  Trading places is funny film - eddie murphy - ...             2   
2  Billy Bob Thornton, super funny, but really ir...             7   
3  7 year old grandson is very into this series r...             0   
4  An oldie, but a goodie in my opinion.  Fun act...             1   

   verified_purchase product_title   category parent_asin  
0                  1          None  Movies_TV  B00HMC4NHK  
1                  1          None  Movies_TV  B08P7ZGLT3  
2                  1          None  Mo

In [ ]:
def get_recommendations_based_on_high_rating(user_reviews_df, high_rating_products_df, tfidf_matrix, asin_to_index, category_df, top_n=5):
    recommendations = []
    reviewed_asins = set(user_reviews_df['parent_asin'])
    for _, review in user_reviews_df.iterrows():
        asin = review['parent_asin']
        if asin not in asin_to_index:
            continue
        idx = asin_to_index[asin]
        cosine_sim = cosine_similarity(tfidf_matrix[idx], tfidf_matrix).flatten()
        similar_indices = cosine_sim.argsort()[::-1][1:top_n+1]

        similar_products_all = category_df.iloc[similar_indices]
        similar_products = similar_products_all[similar_products_all['rating'] == rating_threshold]
        recommendations.append(similar_products)

    if recommendations:
        recommendations_df = pd.concat(recommendations).drop_duplicates()
        recommendations_df = recommendations_df[~recommendations_df['parent_asin'].isin(reviewed_asins)]
        recommendations_df = recommendations_df.head(top_n)
        return recommendations_df
    else:
        return pd.DataFrame()

def get_recommendations_based_on_similarity(user_tfidf, tfidf_matrix, category_df, user_reviews_df, asin_to_index, top_n=5):
    cosine_sim_user = cosine_similarity(user_tfidf, tfidf_matrix).flatten()
    user_asins = user_reviews_df['parent_asin'].tolist()
    user_indices = [asin_to_index[asin] for asin in user_asins if asin in asin_to_index]
    sorted_indices = cosine_sim_user.argsort()[::-1]
    sorted_indices = [idx for idx in sorted_indices if idx not in user_indices]
    top_indices = sorted_indices[:top_n]
    recommended_products = category_df.iloc[top_indices]
    return recommended_products.head(top_n)


def precision_at_k(recommended_asins, val_asins, k=5):
    recommended_top_k = recommended_asins[:k]
    relevant = [asin for asin in recommended_top_k if asin in val_asins]
    precision = len(relevant) / k
    return precision

def recall_at_k(recommended_asins, val_asins, k=5):
    recommended_top_k = recommended_asins[:k]
    relevant = [asin for asin in recommended_top_k if asin in val_asins]
    recall = len(relevant) / len(val_asins) if len(val_asins) > 0 else 0
    return recall

In [ ]:
print(len(val_reviews))
print(len(train_reviews))


165
382


In [ ]:
top_n = 3  # 여기서 추천 개수를 원하는 값(예:50)으로 변경

for target_category in desired_categories:
    print("\n" + "="*50)
    print(f"Category: {target_category}")
    print("="*50)

    category_df = filtered_products_df[filtered_products_df['category'] == target_category].copy()
    category_df['combined_text'] = combine_text(category_df)

    category_train_reviews = train_reviews[train_reviews['category'] == target_category].copy()
    category_val_reviews = val_reviews[val_reviews['category'] == target_category].copy()

    category_train_reviews['combined_text'] = combine_text(category_train_reviews)
    category_val_reviews['combined_text'] = combine_text(category_val_reviews)

    tfidf = TfidfVectorizer(stop_words='english', max_features=5000)
    tfidf_matrix = tfidf.fit_transform(category_df['combined_text'])

    category_df = category_df.reset_index(drop=True)
    category_df['index'] = category_df.index
    asin_to_index = pd.Series(category_df.index, index=category_df['parent_asin']).to_dict()

    print("TF-IDF Matrix Shape:", tfidf_matrix.shape)

    high_rating_products_df = category_df[category_df['rating'] == rating_threshold].copy()
    print(f"Number of products with rating {rating_threshold} in {target_category}: {len(high_rating_products_df)}")

    if len(category_train_reviews) > 0:
        user_query = ' '.join(category_train_reviews['combined_text'].tolist())
        user_tfidf = tfidf.transform([user_query])
    else:
        user_query = ''
        user_tfidf = tfidf.transform([user_query])

    val_asins = category_val_reviews['parent_asin'].unique().tolist()

    # 유사도 기반 추천
    recommended_products_similarity = get_recommendations_based_on_similarity(
        user_tfidf, tfidf_matrix, category_df, category_train_reviews, asin_to_index, top_n=top_n
    )
    recommended_asins_similarity = recommended_products_similarity['parent_asin'].tolist()
    precision_sim = precision_at_k(recommended_asins_similarity, val_asins, k=top_n)
    recall_sim = recall_at_k(recommended_asins_similarity, val_asins, k=top_n)

    print("순수 유사도 기반 추천 성능:")
    print(f"Precision@{top_n}: {precision_sim}, Recall@{top_n}: {recall_sim}")

    # rating 기반 추천
    recommended_products_rating = get_recommendations_based_on_high_rating(
        category_train_reviews, high_rating_products_df, tfidf_matrix, asin_to_index, category_df, top_n=top_n
    )
    if not recommended_products_rating.empty:
        recommended_asins_rating = recommended_products_rating['parent_asin'].tolist()
        precision_rating = precision_at_k(recommended_asins_rating, val_asins, k=top_n)
        recall_rating = recall_at_k(recommended_asins_rating, val_asins, k=top_n)
        print("평가 +유사도(=rating 기반) 추천 성능:")
        print(f"Precision@{top_n}: {precision_rating}, Recall@{top_n}: {recall_rating}")
    else:
        print("평가 +유사도 추천 결과 없음(해당 카테고리에서 rating=5 상품 부재)")



Category: Movies_TV
TF-IDF Matrix Shape: (9998, 5000)
Number of products with rating 5.0 in Movies_TV: 6378
순수 유사도 기반 추천 성능:
Precision@3: 0.3333333333333333, Recall@3: 0.0136986301369863
평가 +유사도(=rating 기반) 추천 성능:
Precision@3: 0.0, Recall@3: 0.0

Category: Grocery_Gourmet_Food
TF-IDF Matrix Shape: (4862, 5000)
Number of products with rating 5.0 in Grocery_Gourmet_Food: 3187
순수 유사도 기반 추천 성능:
Precision@3: 0.6666666666666666, Recall@3: 0.027777777777777776
평가 +유사도(=rating 기반) 추천 성능:
Precision@3: 0.3333333333333333, Recall@3: 0.013888888888888888

Category: Electronics
TF-IDF Matrix Shape: (5471, 5000)
Number of products with rating 5.0 in Electronics: 3581
순수 유사도 기반 추천 성능:
Precision@3: 0.6666666666666666, Recall@3: 0.1
평가 +유사도(=rating 기반) 추천 성능:
Precision@3: 0.0, Recall@3: 0.0


In [ ]:
# 이제 user_review 데이터에 대해서는 성능평가 없이 추천만 수행
print("\n" + "="*50)
print("User Review 기반 추천 (성능평가 없음)")
print("="*50)

# user_review 기반 추천을 위해 사용자 쿼리 생성
# user_reviews_df 전체를 하나의 사용자로 가정 (필요하다면 특정 사용자만 선택 가능)
user_reviews_df['combined_text'] = combine_text(user_reviews_df)

for target_category in desired_categories:
    print("\nCategory:", target_category)
    category_df = filtered_products_df[filtered_products_df['category'] == target_category].copy()
    category_df['combined_text'] = combine_text(category_df)

    tfidf = TfidfVectorizer(stop_words='english', max_features=5000)
    tfidf_matrix = tfidf.fit_transform(category_df['combined_text'])

    category_df = category_df.reset_index(drop=True)
    category_df['index'] = category_df.index
    asin_to_index = pd.Series(category_df.index, index=category_df['parent_asin']).to_dict()

    high_rating_products_df = category_df[category_df['rating'] == rating_threshold].copy()

    user_query_all = ' '.join(user_reviews_df['combined_text'].tolist())
    user_tfidf_all = tfidf.transform([user_query_all])

    # rating 기반 추천 (user_review)
    recommended_products_rating_user = get_recommendations_based_on_high_rating(user_reviews_df, high_rating_products_df, tfidf_matrix, asin_to_index, category_df, top_n=5)
    print("User Review 기반 Rating + 유사도 추천 상품:")
    print(recommended_products_rating_user[['parent_asin', 'product_title', 'category']] if not recommended_products_rating_user.empty else "추천 없음")

    # 유사도 기반 추천 (user_review)
    recommended_products_similarity_user = get_recommendations_based_on_similarity(user_tfidf_all, tfidf_matrix, category_df, user_reviews_df, asin_to_index, top_n=5)
    print("User Review 기반 순수 유사도 추천 상품:")
    print(recommended_products_similarity_user[['parent_asin', 'product_title', 'category']])


User Review 기반 추천 (성능평가 없음)

Category: Movies_TV
User Review 기반 Rating + 유사도 추천 상품:
     parent_asin product_title   category
211   B00AOB5G3Q          None  Movies_TV
210   B084Z2FXBJ          None  Movies_TV
3682  B074QWJM76          None  Movies_TV
5977  B006JN6D6M          None  Movies_TV
5978  B001JGECT4          None  Movies_TV
User Review 기반 순수 유사도 추천 상품:
    parent_asin product_title   category
201  B07ZWWT77T          None  Movies_TV
211  B00AOB5G3Q          None  Movies_TV
210  B084Z2FXBJ          None  Movies_TV
209  B00A65G7X8          None  Movies_TV
205  B00HMC4NHK          None  Movies_TV

Category: Grocery_Gourmet_Food
User Review 기반 Rating + 유사도 추천 상품:
    parent_asin                                      product_title  \
526  B00WI0T4HU  Kraft Original Easy Microwavable Macaroni and ...   
524  B005F7SQKO  Peppadew Peppers - Mild - Value Bundle of 6 (8...   
533  B01MQRSTKC  BUSH'S BEST Canned Reduced Sodium Black Beans ...   
535  B0B75QKL5C  Make Your Day Variety, P